In [17]:
import getpass
import os
import uuid

Tools

In [18]:
# !pip3 install sqlalchemy==2.0.0

In [19]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

# llm = ChatOpenAI(base_url='http://localhost:11434/v1', api_key='ollama', model='mistral:7b-instruct-v0.2-q8_0', temperature=0.5)

from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')
# llm = OllamaFunctions(model='calebfahlgren/natural-functions:Q8_0')

In [20]:
from typing import Annotated, List, Tuple, Union

import matplotlib.pyplot as plt
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langsmith import trace

tavily_tool = TavilySearchResults(max_results=5)


@tool
def scrape_webpages(urls: List[str]) -> str:
    """Use requests and bs4 to scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f'<Document name="{doc.metadata.get("title", "")}">\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

Document writing team tools

In [21]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional

from langchain_experimental.utilities import PythonREPL
from typing_extensions import TypedDict

_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(_TEMP_DIRECTORY.name)


@tool
def create_outline(
    points: Annotated[List[str], "List of main points or sections."],
    file_name: Annotated[str, "File path to save the outline."],
) -> Annotated[str, "Path of the saved outline file."]:
    """Create and save an outline."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        for i, point in enumerate(points):
            file.write(f"{i + 1}. {point}\n")
    return f"Outline saved to {file_name}"


@tool
def read_document(
    file_name: Annotated[str, "File path to save the document."],
    start: Annotated[Optional[int], "The start line. Default is 0"] = None,
    end: Annotated[Optional[int], "The end line. Default is None"] = None,
) -> str:
    """Read the specified document."""
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    if start is not None:
        start = 0
    return "\n".join(lines[start:end])


@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"


@tool
def edit_document(
    file_name: Annotated[str, "Path of the document to be edited."],
    inserts: Annotated[
        Dict[int, str],
        "Dictionary where key is the line number (1-indexed) and value is the text to be inserted at that line.",
    ],
) -> Annotated[str, "Path of the edited document file."]:
    """Edit a document by inserting text at specific line numbers."""

    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()

    sorted_inserts = sorted(inserts.items())

    for line_number, text in sorted_inserts:
        if 1 <= line_number <= len(lines) + 1:
            lines.insert(line_number - 1, text + "\n")
        else:
            return f"Error: Line number {line_number} is out of range."

    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.writelines(lines)

    return f"Document edited and saved to {file_name}"


# Warning: This executes code locally, which can be unsafe when not sandboxed

repl = PythonREPL()


@tool
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."]
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Succesfully executed:\n```python\n{code}\n```\nStdout: {result}"

Helper

In [34]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool

from langgraph.graph import END, StateGraph


def create_agent(
    llm: ChatOllama,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor


def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


def create_team_supervisor(llm: ChatOllama, system_prompt, members) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind(functions=[function_def], function_call={"name": "route"})
        # | JsonOutputFunctionsParser()
    )

Define Agent Teams

Research Team

In [35]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
import functools


# Research team graph state
class ResearchTeamState(TypedDict):
    # A message is added after each team member finishes
    messages: Annotated[List[BaseMessage], operator.add]
    # The team members are tracked so they are aware of
    # the others' skill-sets
    team_members: List[str]
    # Used to route work. The supervisor calls a function
    # that will update this every time it makes a decision
    next: str


llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

search_agent = create_agent(
    llm,
    [tavily_tool],
    "You are a research assistant who can search for up-to-date info using the tavily search engine.",
)
search_node = functools.partial(agent_node, agent=search_agent, name="Search")

research_agent = create_agent(
    llm,
    [scrape_webpages],
    "You are a research assistant who can scrape specified urls for more detailed information using the scrape_webpages function.",
)
research_node = functools.partial(agent_node, agent=research_agent, name="Web Scraper")

supervisor_agent = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  Search, Web Scraper. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Search", "Web Scraper"],
)

graphs

In [36]:
research_graph = StateGraph(ResearchTeamState)
research_graph.add_node("Search", search_node)
research_graph.add_node("Web Scraper", research_node)
research_graph.add_node("supervisor", supervisor_agent)

# Define the control flow
research_graph.add_edge("Search", "supervisor")
research_graph.add_edge("Web Scraper", "supervisor")
research_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {"Search": "Search", "Web Scraper": "Web Scraper", "FINISH": END},
)


research_graph.set_entry_point("supervisor")
chain = research_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results


research_chain = enter_chain | chain

Document Writing Team

In [37]:
import operator
from pathlib import Path


# Document writing team graph state
class DocWritingState(TypedDict):
    # This tracks the team's conversation internally
    messages: Annotated[List[BaseMessage], operator.add]
    # This provides each worker with context on the others' skill sets
    team_members: str
    # This is how the supervisor tells langgraph who to work next
    next: str
    # This tracks the shared directory state
    current_files: str


# This will be run before each worker agent begins work
# It makes it so they are more aware of the current state
# of the working directory.
def prelude(state):
    written_files = []
    if not WORKING_DIRECTORY.exists():
        WORKING_DIRECTORY.mkdir()
    try:
        written_files = [
            f.relative_to(WORKING_DIRECTORY) for f in WORKING_DIRECTORY.rglob("*")
        ]
    except:
        pass
    if not written_files:
        return {**state, "current_files": "No files written."}
    return {
        **state,
        "current_files": "\nBelow are files your team has written to the directory:\n"
        + "\n".join([f" - {f}" for f in written_files]),
    }


llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

doc_writer_agent = create_agent(
    llm,
    [write_document, edit_document, read_document],
    "You are an expert writing a research document.\n"
    # The {current_files} value is populated automatically by the graph state
    "Below are files currently in your directory:\n{current_files}",
)
# Injects current directory working state before each call
context_aware_doc_writer_agent = prelude | doc_writer_agent
doc_writing_node = functools.partial(
    agent_node, agent=context_aware_doc_writer_agent, name="Doc Writer"
)

note_taking_agent = create_agent(
    llm,
    [create_outline, read_document],
    "You are an expert senior researcher tasked with writing a paper outline and"
    " taking notes to craft a perfect paper.{current_files}",
)
context_aware_note_taking_agent = prelude | note_taking_agent
note_taking_node = functools.partial(
    agent_node, agent=context_aware_note_taking_agent, name="Note Taker"
)

chart_generating_agent = create_agent(
    llm,
    [read_document, python_repl],
    "You are a data viz expert tasked with generating charts for a research project."
    "{current_files}",
)
context_aware_chart_generating_agent = prelude | chart_generating_agent
chart_generating_node = functools.partial(
    agent_node, agent=context_aware_note_taking_agent, name="Chart Generator"
)

doc_writing_supervisor = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following workers:  {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Doc Writer", "Note Taker", "Chart Generator"],
)

graph

In [38]:
# Create the graph here:
# Note that we have unrolled the loop for the sake of this doc
authoring_graph = StateGraph(DocWritingState)
authoring_graph.add_node("Doc Writer", doc_writing_node)
authoring_graph.add_node("Note Taker", note_taking_node)
authoring_graph.add_node("Chart Generator", chart_generating_node)
authoring_graph.add_node("supervisor", doc_writing_supervisor)

# Add the edges that always occur
authoring_graph.add_edge("Doc Writer", "supervisor")
authoring_graph.add_edge("Note Taker", "supervisor")
authoring_graph.add_edge("Chart Generator", "supervisor")

# Add the edges where routing applies
authoring_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Doc Writer": "Doc Writer",
        "Note Taker": "Note Taker",
        "Chart Generator": "Chart Generator",
        "FINISH": END,
    },
)

authoring_graph.set_entry_point("supervisor")
chain = research_graph.compile()


# The following functions interoperate between the top level graph state
# and the state of the research sub-graph
# this makes it so that the states of each graph don't get intermixed
def enter_chain(message: str, members: List[str]):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ", ".join(members),
    }
    return results


# We re-use the enter/exit functions to wrap the graph
authoring_chain = (
    functools.partial(enter_chain, members=authoring_graph.nodes)
    | authoring_graph.compile()
)

Layers

In [39]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage


llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K')

supervisor_node = create_team_supervisor(
    llm,
    "You are a supervisor tasked with managing a conversation between the"
    " following teams: {team_members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH.",
    ["Research team", "Paper writing team"],
)

Top Graph

In [40]:
# Top-level graph state
class State(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next: str


def get_last_message(state: State) -> str:
    return state["messages"][-1].content


def join_graph(response: dict):
    return {"messages": [response["messages"][-1]]}


# Define the graph.
super_graph = StateGraph(State)
# First add the nodes, which will do the work
super_graph.add_node("Research team", get_last_message | research_chain | join_graph)
super_graph.add_node(
    "Paper writing team", get_last_message | authoring_chain | join_graph
)
super_graph.add_node("supervisor", supervisor_node)

# Define the graph connections, which controls how the logic
# propagates through the program
super_graph.add_edge("Research team", "supervisor")
super_graph.add_edge("Paper writing team", "supervisor")
super_graph.add_conditional_edges(
    "supervisor",
    lambda x: x["next"],
    {
        "Paper writing team": "Paper writing team",
        "Research team": "Research team",
        "FINISH": END,
    },
)
super_graph.set_entry_point("supervisor")
super_graph = super_graph.compile()

In [41]:
super_graph

CompiledGraph(nodes={'Research team': ChannelInvoke(bound=RunnableLambda(...)
| RunnableLambda(get_last_message)
| RunnableLambda(enter_chain)
| CompiledGraph(nodes={'Search': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Search', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Search:inbox'}, triggers=['Search:inbox']), 'Web Scraper': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Web Scraper', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Web Scraper:inbox'}, triggers=['Web Scraper:inbox']), 'supervisor': ChannelInvoke(bound=RunnableLambda(...)
  | ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.Lis

In [42]:
from langchain.globals import set_debug
set_debug(True)
from langchain.globals import set_verbose
# set_verbose(True)

In [43]:
for s in super_graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Write a brief research report on the Recent trends in Generative AI. Include a chart."
            )
        ],
    },
    {"recursion_limit": 150},
):
    if "__end__" not in s:
        print(s)
        print("---")
        

[chain/start] [1:chain:LangGraph] Entering Chain run with input:
{
  "input": ""
}
[pregel/step] Starting step 0 with 1 task. Next tasks:
- __start__({'messages': [HumanMessage(content='Write a brief research report on the Recent trends in Generative AI. Include a chart.')]})
[chain/start] [1:chain:LangGraph > 2:chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:LangGraph > 2:chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:LangGraph > 3:chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:LangGraph > 3:chain:__start__] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:LangGraph > 4:chain:__start__] Entering Chain run with input:
[inputs]
[chain/end] [1:chain:LangGraph > 4:chain:__start__] s] Exiting Chain run with output:
[outputs]
[pregel/checkpoint] Finishing step 0. Channel values:
{'__start__': {...},
 '__start__:inbox': {...},
 <ReservedChannels.is_last_step: 'is_last_step'>: F

ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q6_K output: {
  "tool": "Research team",
  "tool_input": {
    "next": "Research"
  }
}

In [16]:
for s in super_graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Write a brief research report on the Recent trends in Generative AI. Include a chart."
            )
        ],
    },
    {"recursion_limit": 150},
):
    if "__end__" not in s:
        print(s)
        print("---")
        

ValueError: Ollama call failed with status code 400. Details: {"error":"exception server shutting down"}

In [ ]:
ValueError: Failed to parse a function call from mistral:7b-instruct-v0.2-q8_0 output: {
  "tool": "Research team",
  "tool_input": {
    "next": "Research"
  }
}

In [17]:
out = {
  "generations": [
    [
      {
        "text": "{\n  \"tool\": \"Research team\",\n  \"tool_input\": {\n    \"next\": \"Research\"\n  }\n}",
        "generation_info": {
          "model": "mistral:7b-instruct-v0.2-q8_0",
          "created_at": "2024-03-15T13:17:14.712580891Z",
          "message": {
            "role": "assistant",
            "content": ""
          },
          "done": 'true',
          "total_duration": 6579789112,
          "load_duration": 3031967062,
          "prompt_eval_count": 353,
          "prompt_eval_duration": 825254000,
          "eval_count": 34,
          "eval_duration": 2720940000
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\n  \"tool\": \"Research team\",\n  \"tool_input\": {\n    \"next\": \"Research\"\n  }\n}"
          }
        }
      }
    ]
  ],
  "llm_output": 'null',
  "run": 'null'
}

In [19]:
parser = JsonOutputFunctionsParser()

In [13]:
parser.invoke(out)

NameError: name 'parser' is not defined

In [23]:
# supervisor_node.format(member=['Research team', 'Paper writing team'])

AttributeError: 'RunnableSequence' object has no attribute 'format'

In [ ]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(base_url='http://localhost:11434/v1', api_key='ollama', model='phi:2.7b-chat-v2-q8_0', temperature=0.5)

In [ ]:
# llm.invoke('mine was bit bad')

In [ ]:
# super_graph.get_graph()

In [ ]:
# from IPython.display import Image

# Image(super_graph.get_graph().draw_png())

Graph(nodes={'__start__': Node(id='__start__', data=<class 'pydantic.v1.main.LangGraphInput'>), '__end__': Node(id='__end__', data=<class 'pydantic.v1.main.LangGraphOutput'>), 'Research team': Node(id='Research team', data=RunnableLambda(get_last_message)
| RunnableLambda(enter_chain)
| CompiledGraph(nodes={'Search': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Search', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Search:inbox'}, triggers=['Search:inbox']), 'Web Scraper': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Web Scraper', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Web Scraper:inbox'}, triggers=['Web Scraper:inbox']), 'supervisor': ChannelInvoke(bound=RunnableLambda(...)
  | ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Search', 'Web Scraper']", 'team_members': 'Search, Web Scraper'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a supervisor tasked with managing a conversation between the following workers:  Search, Web Scraper. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}'))])
  | RunnableBinding(bound=OllamaFunctions(llm=ChatOllama(model='mistral:7b-instruct-v0.2-q8_0', format='json'), tool_system_prompt_template='You have access to the following tools:\n\n{tools}\n\nYou must always select one of the above tools and respond with only a JSON object matching the following schema:\n\n{{\n  "tool": <name of the selected tool>,\n  "tool_input": <parameters for the selected tool, matching the tool\'s JSON schema>\n}}\n'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Search', 'Web Scraper']}]}}, 'required': ['next']}}], 'function_call': 'route'})
  | JsonOutputFunctionsParser()
  | ChannelWrite(channels=[('supervisor', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'supervisor:inbox'}, triggers=['supervisor:inbox']), 'Search:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: 'Search'}, triggers=['Search']), 'Web Scraper:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: 'Web Scraper'}, triggers=['Web Scraper']), 'supervisor:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | RunnableLambda(runnable), config={'tags': ['langsmith:hidden']}, channels={None: 'supervisor'}, triggers=['supervisor']), '__start__': ChannelInvoke(bound=ChannelWrite(channels=[('__start__', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False)]), config={'tags': ['langsmith:hidden']}, channels={None: '__start__:inbox'}, triggers=['__start__:inbox']), '__start__:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: '__start__'}, triggers=['__start__'])}, channels={'messages': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7fb1a41801f0>, 'team_members': <langgraph.channels.last_value.LastValue object at 0x7fb1a4180550>, 'next': <langgraph.channels.last_value.LastValue object at 0x7fb1a41817e0>, 'Search:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a4370670>, 'Web Scraper:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a4183f70>, 'supervisor:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a41823b0>, '__start__:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a41821a0>, 'Search': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a4180610>, 'Web Scraper': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a41805b0>, 'supervisor': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a4182110>, '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a41834c0>, '__end__': <langgraph.channels.last_value.LastValue object at 0x7fb1a4181c90>, <ReservedChannels.is_last_step: 'is_last_step'>: <langgraph.channels.last_value.LastValue object at 0x7fb1a41800d0>}, output='__end__', hidden=['Search:inbox', 'Web Scraper:inbox', 'supervisor:inbox', '__start__', 'messages', 'team_members', 'next'], input='__start__:inbox', graph=<langgraph.graph.state.StateGraph object at 0x7fb1a4183850>)
| RunnableLambda(join_graph)), 'Paper writing team': Node(id='Paper writing team', data=RunnableLambda(get_last_message)
| RunnableLambda(...)
| CompiledGraph(nodes={'Doc Writer': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Doc Writer', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False), ('current_files', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Doc Writer:inbox'}, triggers=['Doc Writer:inbox']), 'Note Taker': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Note Taker', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False), ('current_files', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Note Taker:inbox'}, triggers=['Note Taker:inbox']), 'Chart Generator': ChannelInvoke(bound=RunnableLambda(...)
  | RunnableLambda(...)
  | ChannelWrite(channels=[('Chart Generator', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False), ('current_files', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'Chart Generator:inbox'}, triggers=['Chart Generator:inbox']), 'supervisor': ChannelInvoke(bound=RunnableLambda(...)
  | ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Doc Writer', 'Note Taker', 'Chart Generator']", 'team_members': 'Doc Writer, Note Taker, Chart Generator'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['team_members'], template='You are a supervisor tasked with managing a conversation between the following workers:  {team_members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}'))])
  | RunnableBinding(bound=ChatOllama(model='mistral:7b-instruct-v0.2-q8_0'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Doc Writer', 'Note Taker', 'Chart Generator']}]}}, 'required': ['next']}}], 'function_call': 'route'})
  | JsonOutputFunctionsParser()
  | ChannelWrite(channels=[('supervisor', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False), ('current_files', RunnableLambda(...), False)]), config={'tags': []}, channels={None: 'supervisor:inbox'}, triggers=['supervisor:inbox']), 'Doc Writer:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: 'Doc Writer'}, triggers=['Doc Writer']), 'Note Taker:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: 'Note Taker'}, triggers=['Note Taker']), 'Chart Generator:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: 'Chart Generator'}, triggers=['Chart Generator']), 'supervisor:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | RunnableLambda(runnable), config={'tags': ['langsmith:hidden']}, channels={None: 'supervisor'}, triggers=['supervisor']), '__start__': ChannelInvoke(bound=ChannelWrite(channels=[('__start__', None, False), ('messages', RunnableLambda(...), False), ('team_members', RunnableLambda(...), False), ('next', RunnableLambda(...), False), ('current_files', RunnableLambda(...), False)]), config={'tags': ['langsmith:hidden']}, channels={None: '__start__:inbox'}, triggers=['__start__:inbox']), '__start__:edges': ChannelInvoke(bound=RunnableLambda(_read)
  | ChannelWrite(channels=[('supervisor:inbox', None, False)]), config={'tags': ['langsmith:hidden']}, channels={None: '__start__'}, triggers=['__start__'])}, channels={'messages': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7fb1a43b8e20>, 'team_members': <langgraph.channels.last_value.LastValue object at 0x7fb1a43b89d0>, 'next': <langgraph.channels.last_value.LastValue object at 0x7fb1a43b8e80>, 'current_files': <langgraph.channels.last_value.LastValue object at 0x7fb1a43b86d0>, 'Doc Writer:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a43b9360>, 'Note Taker:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a43b8cd0>, 'Chart Generator:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a43b8fd0>, 'supervisor:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a425bd90>, '__start__:inbox': <langgraph.channels.any_value.AnyValue object at 0x7fb1a425aef0>, 'Doc Writer': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a425bb20>, 'Note Taker': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a425b400>, 'Chart Generator': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a425bc70>, 'supervisor': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a425ae90>, '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7fb1a425b160>, '__end__': <langgraph.channels.last_value.LastValue object at 0x7fb1a425bc40>, <ReservedChannels.is_last_step: 'is_last_step'>: <langgraph.channels.last_value.LastValue object at 0x7fb1a425b6a0>}, output='__end__', hidden=['Doc Writer:inbox', 'Note Taker:inbox', 'Chart Generator:inbox', 'supervisor:inbox', '__start__', 'messages', 'team_members', 'next', 'current_files'], input='__start__:inbox', graph=<langgraph.graph.state.StateGraph object at 0x7fb1a43b8bb0>)
| RunnableLambda(join_graph)), 'supervisor': Node(id='supervisor', data=ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'options': "['FINISH', 'Research team', 'Paper writing team']", 'team_members': 'Research team, Paper writing team'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['team_members'], template='You are a supervisor tasked with managing a conversation between the following teams: {team_members}. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.')), MessagesPlaceholder(variable_name='messages'), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['options'], template='Given the conversation above, who should act next? Or should we FINISH? Select one of: {options}'))])
| RunnableBinding(bound=ChatOllama(model='mistral:7b-instruct-v0.2-q8_0'), kwargs={'functions': [{'name': 'route', 'description': 'Select the next role.', 'parameters': {'title': 'routeSchema', 'type': 'object', 'properties': {'next': {'title': 'Next', 'anyOf': [{'enum': ['FINISH', 'Research team', 'Paper writing team']}]}}, 'required': ['next']}}], 'function_call': 'route'})
| JsonOutputFunctionsParser()), 'supervisor_<lambda>': Node(id='supervisor_<lambda>', data=RunnableLambda(runnable))}, edges=[Edge(source='Research team', target='supervisor', data=None), Edge(source='Paper writing team', target='supervisor', data=None), Edge(source='supervisor', target='supervisor_<lambda>', data=None), Edge(source='supervisor_<lambda>', target='Paper writing team', data='Paper writing team'), Edge(source='supervisor_<lambda>', target='Research team', data='Research team'), Edge(source='supervisor_<lambda>', target='__end__', data='FINISH'), Edge(source='__start__', target='supervisor', data=None)])
